# LangChain 核心模块 Agent - Self ask with search

- Google 搜索对接
    - 第三方平台：https://serpapi.com
    - LangChain API 封装：`SerpAPIWrapper`
- LangChain Agent 模块使用
    - Agent 类型：`SELF_ASK_WITH_SEARCH`
    - Agent 实例化：`initialize_agent` 方法
 
LangChain 内置 Agent类型详见：https://python.langchain.com/docs/modules/agents/agent_types/

In [1]:
%%capture --no-stderr
# %pip install -U langchain

以下`SERPAPI_API_KEY`仅为示例，请访问 https://serpapi.com 注册账号并替换为自己的 `API_KEY`（每月100次免费调用）

```python
import os

# 更换为自己的 Serp API KEY
os.environ["SERPAPI_API_KEY"] = "xxx"
```

建议将其设置为环境变量，而不是在代码中显式明文设置

In [2]:
from langchain_openai import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)

In [3]:
# 实例化查询工具
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [4]:
# 实例化 SELF_ASK_WITH_SEARCH Agent
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

C:\Users\huangaf\AppData\Local\Temp\ipykernel_16040\2069300735.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  self_ask_with_search = initialize_agent(


In [5]:
# 实际运行 Agent，查询问题（正确）
self_ask_with_search.invoke(
    "成都举办的大运会是第几届大运会？2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: 第几届大运会是在哪一年举办的？
Intermediate answer: [' entity_type: related_questions.', '有14个运动项目为必须举办，另可由东道主自行确定最多3个可选项目。 2003年大邱大运会有174个国家参赛，为参赛国家最多的一次。 2015光州大运会有1万2,885名运动员参赛，为参 ...', '大运会始办于1959年，其前身为国际大学生运动会，从1959年起至今已举办31届。 ... 大运会所有必选竞赛项目全部是奥运项目，竞赛管理由国际单项体育组织按照奥运标准实施。', '-- 第二十一届世界大学生运动会-- 二战之后，从1946年，一个新组织成立了，“世界学生联合会”（U.I.E），1947年战后运动会在巴黎召开，来自17个国家的代表团共计800余名运动员参加 ...', '-- 第二十一届世界大学生运动会-- 60年代FISU所承办的大运会在组织形式,参与程度及比赛项目水平上都得到了很大提高,1977年的大运会更进一步推进了FISU运动会，1979年在墨西 ...', '这届大运会将于8月8日到18日在泰国首都曼谷举行。全世界153个国家和地区的一万两千余名大学生运动员将参赛，争夺18个大项236枚金牌。', '8月8日上午，大运会组委会新闻部组织60多名主要媒体记者纷纷云集清华园跳水馆参加了第二十一届世界大学生运动会新闻发布会。 清华大学副校长，第二十一届世界大学生 ...', '第二届大运会由国家教委、国家体委和共青团中央联合主办，辽宁省大连市承办，历时7天（1986年8月3日至8月9日）。 全国29个省、市、自治区的2228名运动员参加了 ...', '首先，这是一次来之不易的大运会。自2018年12月获得大运会举办权以来，四川省、成都市做了大量工作。特别是过去三年，新冠疫情给 ...', '2011年夏季世界大学生运动会是第26届世界大学生运动会，于2011年8月12日至2011年8月23日在中国深圳市举办，此次大运会的安保措施堪比奥运会。这是中国第2次举办夏季世界 ...', '"第21届世界大学生运动会"将于8月22日到9月1日在北京举行。这是人类迈入新千年、新世纪的第一个世界大学生运

{'input': '成都举办的大运会是第几届大运会？2023年大运会举办地在哪里？', 'output': '第二十一届世界大学生运动会'}

In [6]:
# 实际运行 Agent，查询问题
self_ask_with_search.invoke(
    "2023年大运会举办地在哪里？成都举办的大运会是第几届大运会？"
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: 2023年大运会是第几届大运会？
Intermediate answer: 2023成都第31届世界大学生运动会开幕式& 2023 The 31 FISU World University Games - Opening Ceremony(4K HDR) - YouTube.
Follow up: 成都是第几届大运会举办地？
Intermediate answer: 成都第31届世界大学生夏季运动会开幕式直播_新华网 第31届世界大学生夏季运动会开幕式7月28日晚在四川省成都市举行。
So the final answer is: 成都是第31届大运会举办地。

> Finished chain.


{'input': '2023年大运会举办地在哪里？成都举办的大运会是第几届大运会？', 'output': '成都是第31届大运会举办地。'}

In [7]:
# Reason-only 正确：启发式 Prompt（猜测是大运会新闻报道数据给到了 gpt-3.5-turbo-instruct 模型）
print(llm.invoke("成都举办的大运会是第几届大运会？"))



成都举办的大运会是第31届大运会。


In [9]:
# Reason-only 错误：非启发式 Prompt（容易出现事实类错误，未结合 Web Search 工具）
print(llm.invoke("2023年大运会举办地在哪里？"))



2023年大运会的举办地将在中国的重庆市。


#### 使用 GPT-4 作为大语言模型实现更优的 ReAct 范式

In [10]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [11]:
self_ask_with_search_chat = initialize_agent(
    tools, chat_model, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [12]:
# GPT-4 based ReAct 答案（正确）
self_ask_with_search_chat.invoke(
     "成都举办的大运会是第几届大运会？2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
Yes.  
Follow up: 成都举办的大运会是第几届大运会？  
Intermediate answer: 成都第31届世界大学生夏季运动会开幕式直播_新华网 第31届世界大学生夏季运动会开幕式7月28日晚在四川省成都市举行。
Follow up: 2023年大运会举办地在哪里？  
Intermediate answer: 2023年7月28日晚，共享大运盛会，共谱青春华章。 第三十一届世界大学生夏季运动会在四川省成都市隆重开幕。 国家主席习近平出席开幕式并宣布本届大运会开幕。
So the final answer is: 第31届大运会，举办地在四川省成都市。

> Finished chain.


{'input': '成都举办的大运会是第几届大运会？2023年大运会举办地在哪里？', 'output': '第31届大运会，举办地在四川省成都市。'}